# Haircut shop Query answering using Finetuned LLM

In [25]:
queries = [
    "When is the least crowded time for a haircut?",
    "What types of haircuts do you offer?",
    "Do you provide home service for haircuts?",
    "How much does a fade haircut cost?",
    "Can I book an appointment for Sunday?",
    "Who is your best barber?",
    "Do you offer discounts for students?",
    "How long does a haircut take?",
    "Can I cancel my appointment?",
    "Do you offer beard trimming?",
    "What are your working hours?",
    "Do I need to make an appointment, or do you accept walk-ins?",
    "Can I request a specific barber?",
    "Do you offer hair coloring services?",
    "Is there an additional charge for long hair?",
    "Do you cut children's hair?",
    "Do you offer senior citizen discounts?",
    "What payment methods do you accept?",
    "Can I tip my barber using a card payment?",
    "Do you sell hair care products?",
    "How often should I get a haircut?",
    "Can I bring my own hair products for the service?",
    "Do you offer scalp treatments?",
    "Do you offer wedding or special event hair styling?",
    "Do you provide hair washing before the cut?",
    "Can I get a haircut if I have dandruff or scalp issues?",
    "Do you offer keratin or hair smoothing treatments?",
    "What is the price of a basic men's haircut?",
    "Do you offer women's haircuts as well?",
    "Do you have loyalty programs or membership discounts?",
    "Can I reschedule my appointment?",
    "Do you offer head massages?",
    "Do you provide eyebrow trimming services?",
    "What safety measures do you follow for hygiene?",
    "Can I get a consultation before deciding on a haircut?",
    "Do you have a waiting area with Wi-Fi?",
    "Can I bring my pet to the salon?",
    "Do you offer kids' haircut packages?",
    "What happens if I'm late for my appointment?",
    "Can I donate my hair after a haircut?",
    "Do you offer hairstyling tutorials or workshops?",
    "Do you provide services for special needs customers?",
    "How do I book an appointment online?",
    "Do you have a mobile app for booking?",
    "Can I get a refund if I don’t like my haircut?",
    "Can I get a haircut if I have recently dyed my hair?",
    "Do you offer perms or curly hair treatments?",
    "Do you provide straight razor shaves?",
    "Can I book a haircut for multiple people at once?",
    "Do you offer seasonal promotions or holiday discounts?",
    "What is your cancellation policy?"
]

responses = [
    "Mornings on weekdays (Monday to Thursday) are usually less busy.",
    "We offer buzz cuts, fades, crew cuts, undercuts, and many more styles.",
    "Yes, we offer home services. You can book through our website.",
    "A fade haircut costs $30.",
    "Yes, we are open on Sundays. What time would you prefer?",
    "John is our most experienced barber with 10 years of experience.",
    "Yes, students get a 15% discount on all services.",
    "A standard haircut takes about 30-45 minutes.",
    "Yes, you can cancel your appointment 24 hours in advance.",
    "Yes, beard trimming is available for $15.",
    "We are open from 9 AM to 8 PM, Monday to Saturday, and 10 AM to 6 PM on Sundays.",
    "Walk-ins are welcome, but appointments are recommended for guaranteed service.",
    "Yes, you can request a specific barber while booking your appointment.",
    "Yes, we provide hair coloring services starting at $50.",
    "Yes, long hair services may have an additional charge depending on the length.",
    "Yes, we cut children's hair and have a special area for kids.",
    "Yes, senior citizens get a 10% discount on all services.",
    "We accept cash, credit/debit cards, and digital wallets like Apple Pay and Google Pay.",
    "Yes, you can tip your barber using a card or cash.",
    "Yes, we sell shampoos, conditioners, and styling products from top brands.",
    "It depends on your hair type, but generally, every 4-6 weeks is recommended.",
    "Yes, you can bring your own hair products if you prefer.",
    "Yes, we offer scalp treatments for dry or oily scalp issues.",
    "Yes, we provide wedding and event hairstyling upon prior booking.",
    "Yes, we wash your hair before the cut unless requested otherwise.",
    "Yes, but we recommend consulting with our barber to check the condition of your scalp.",
    "Yes, we offer keratin treatments for frizz-free and smooth hair.",
    "A basic men's haircut starts at $25.",
    "Yes, we offer women's haircuts, including layers, bobs, and more.",
    "Yes, we have a loyalty program where you earn points for every service.",
    "Yes, you can reschedule your appointment up to 12 hours in advance.",
    "Yes, we offer head massages as an add-on service.",
    "Yes, we provide eyebrow trimming for $10.",
    "We use sterilized tools, clean capes, and follow strict sanitation protocols.",
    "Yes, we offer free consultations to discuss hairstyle options.",
    "Yes, we have a comfortable waiting area with free Wi-Fi and magazines.",
    "We love pets, but for hygiene reasons, only service animals are allowed.",
    "Yes, we offer special kids' haircut packages with fun styling options.",
    "If you're more than 10 minutes late, we may need to reschedule your appointment.",
    "Yes, we accept hair donations for charities like Locks of Love.",
    "Yes, we occasionally hold hairstyling workshops and tutorials.",
    "Yes, we provide services tailored for customers with special needs.",
    "You can book an appointment through our website or by calling us.",
    "Yes, we have a mobile app where you can book and manage appointments.",
    "If you're unsatisfied with your haircut, we offer free touch-ups within 7 days.",
    "Yes, but we recommend waiting at least 2 weeks after dyeing before cutting.",
    "Yes, we provide perms, wave styling, and curly hair treatments.",
    "Yes, we offer straight razor shaves for a classic barber experience.",
    "Yes, you can book group appointments for families or friends.",
    "Yes, we run promotions during holidays and special events.",
    "Cancellations must be made at least 24 hours in advance to avoid a fee."
]

print("Length of queries: ", len(queries))
print("Length of responses: ", len(responses))

Length of queries:  51
Length of responses:  51


In [26]:
import pandas as pd
from datasets import Dataset

# Format data as prompt-response
df = pd.DataFrame({
    "text": ["Question: " + q + "\nAnswer: " + r for q, r in zip(queries, responses)]
})

# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Required for GPT-2 padding

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Tokenize dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [28]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

# Load model
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))  # Update vocab size if needed

# Training setup
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    num_train_epochs=10,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none"
)

# Language modeling collator (no masked LM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

# Start fine-tuning
trainer.train()

# Save model
trainer.save_model("./fine_tuned_gpt2")
tokenizer.save_pretrained("./fine_tuned_gpt2")


Step,Training Loss
10,2.490000
20,1.677900
30,1.502300
40,1.281100
50,1.033400
60,0.912000
70,0.794200
80,0.683100
90,0.677000
100,0.571200


('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json',
 './fine_tuned_gpt2/tokenizer.json')

In [29]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load fine-tuned model
model_path = "./fine_tuned_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [30]:
def generate_response_fixed(question, max_length=100):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_length=max_length,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=2.0,
            do_sample=True,
            num_beams=4,
            early_stopping=True
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Post-process to clean repeated or extra responses
    if "Answer:" in response:
        response = response.split("Answer:")[1].strip()
        response = response.split("\n")[0].strip()

    return response


In [35]:
question = "Do you provide services for special needs customers?"
print("🛎️ Question:", question)
print("🤖 Response:", generate_response_fixed(question))


🛎️ Question: Do you provide services for special needs customers?
🤖 Response: Yes, we provide services tailored for customers with special needs. You can book through our website or by calling us.


In [36]:
question = "Is there an additional charge for long hair?"
print("🛎️ Question:", question)
print("🤖 Response:", generate_response_fixed(question))

🛎️ Question: Is there an additional charge for long hair?
🤖 Response: Yes, long hair services may have an additional charge depending on the length. Long hair services may have an additional charge depending on the length. What services do you provide? Long hair services may have an additional charge depending on the length. What services do you provide? Long hair services may have an additional charge depending on the length. What services do you provide? Long hair services may have an additional charge depending on the length. What


In [37]:
question = "Who is your best barber?"
print("🛎️ Question:", question)
print("🤖 Response:", generate_response_fixed(question))

🛎️ Question: Who is your best barber?
🤖 Response: John is our most experienced barber with more than 30 years of experience. We have a wide selection of styles to choose from, including classic cuts, undercuts, and many more. You can also check out our website to find out more about our services.
